## Intent classifier trainer

* Dataset: student_queries.csv file with synthetic messages, each labeled with one of the intent categories, such as: Course Information, Enrollment / Course Registration, Withdrawal or Drop Course, Scholarship/Financial Aid, etc.
* Task: Single-label classification to predict the intent behind the student's query.
* Model: Use BertForSequenceClassification, fine-tuned on the intent dataset.
* Goal: To determine the purpose of a query.
* The trained model is saved to models/intentClassifier.

In [1]:
intent_labels = [
        "Course Information",
        "Enrollment / Course Registration",
        "Withdrawal or Drop Course",
        "Access Issues (portal/login)",
        "Technical Support",
        "Tuition/Fees Inquiry",
        "Scholarship/Financial Aid",
        "Mental Health Concerns",
        "Stress or Burnout",
        "Bullying or Harassment",
        "Administrative Support",
        "Campus Facilities",
        "Housing/Accommodation",
        "Extracurricular Activities",
        "General Complaint"
    ]

In [2]:
import pandas as pd

label_to_id = {label: idx for idx, label in enumerate(intent_labels)}
id_to_label = {idx: label for idx, label in enumerate(intent_labels)}

df = pd.read_csv("../../data/student_queries.csv")

# Encode labels
df["label"] = df["intent"].map(label_to_id)
df = df.dropna(subset=["label"])

# convert label to int (from float due to NaN)
df["label"] = df["label"].astype(int)

In [3]:
df.head()

,datetime,student,question,intent,is_distressed,label
0,2025-07-04 14:07:23,Shannon Austin,What topics will be covered in the AI course?,Course Information,False,0
1,2025-07-04 16:13:28,Stephanie Calhoun,I'm feeling really overwhelmed lately.,Mental Health Concerns,True,7
2,2025-07-04 16:19:28,Kevin Garcia,Who do I contact for transcript requests?,Administrative Support,False,10
3,2025-07-04 16:55:20,Lisa Duran,I have a complaint about the cafeteria service.,General Complaint,True,14
4,2025-07-04 17:08:35,Jeff Rangel,Who do I contact for transcript requests?,Administrative Support,False,10


In [4]:
from sklearn.model_selection import train_test_split
from datasets import Dataset

# Train-test split
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df["label"], random_state=42)

# Convert to Hugging Face Dataset format
train_dataset = Dataset.from_pandas(train_df[["question", "label"]])
test_dataset = Dataset.from_pandas(test_df[["question", "label"]])

c:\Source\conestoga\misc\CSCN8010_Project-Student_Success_Advisors\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import torch

# Tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize(batch):
    return tokenizer(batch["question"], padding=True, truncation=True)

train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

# Set format for PyTorch
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])

# Model setup
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=len(intent_labels)
)

# Metrics
def compute_metrics(pred):
    logits, labels = pred
    preds = logits.argmax(axis=1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted"),
        "precision": precision_score(labels, preds, average="weighted"),
        "recall": recall_score(labels, preds, average="weighted"),
    }

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_f1",
    greater_is_better=True,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=100,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

W0805 09:01:27.407330 14064 Lib\site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
Map: 100%|██████████| 615/615 [00:00<00:00, 5321.39 examples/s]
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Source\conestoga\misc\CSCN8010_Project-Student_Success_Advisors\.venv\lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [6]:
# Train the model
trainer.train()

# Evaluate on test set
trainer.evaluate()

  0%|          | 0/1540 [00:00<?, ?it/s]c:\Source\conestoga\misc\CSCN8010_Project-Student_Success_Advisors\.venv\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
  6%|▋         | 100/1540 [03:18<1:09:46,  2.91s/it]

{'loss': 1.8448, 'grad_norm': 8.134522438049316, 'learning_rate': 1.8701298701298704e-05, 'epoch': 0.65}


                                                    
 10%|█         | 154/1540 [06:25<1:08:32,  2.97s/it]

{'eval_loss': 0.3557010889053345, 'eval_accuracy': 0.9934959349593496, 'eval_f1': 0.9934746332514641, 'eval_precision': 0.9936120789779326, 'eval_recall': 0.9934959349593496, 'eval_runtime': 24.7411, 'eval_samples_per_second': 24.857, 'eval_steps_per_second': 0.404, 'epoch': 1.0}


c:\Source\conestoga\misc\CSCN8010_Project-Student_Success_Advisors\.venv\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
 13%|█▎        | 200/1540 [08:55<1:09:02,  3.09s/it]

...

{'eval_loss': 0.020941605791449547, 'eval_accuracy': 0.9934959349593496, 'eval_f1': 0.993493515508241, 'eval_precision': 0.9937246216315984, 'eval_recall': 0.9934959349593496, 'eval_runtime': 12.9016, 'eval_samples_per_second': 47.668, 'eval_steps_per_second': 0.775, 'epoch': 4.0}


 40%|████      | 616/1540 [30:14<45:22,  2.95s/it]
c:\Source\conestoga\misc\CSCN8010_Project-Student_Success_Advisors\.venv\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'train_runtime': 1814.8303, 'train_samples_per_second': 13.555, 'train_steps_per_second': 0.849, 'train_loss': 0.41116700860877314, 'epoch': 4.0}


100%|██████████| 10/10 [00:22<00:00,  2.22s/it]


{'eval_loss': 0.04386812075972557,
 'eval_accuracy': 0.9967479674796748,
 'eval_f1': 0.9967474837622093,
 'eval_precision': 0.9968253968253968,
 'eval_recall': 0.9967479674796748,
 'eval_runtime': 24.7237,
 'eval_samples_per_second': 24.875,
 'eval_steps_per_second': 0.404,
 'epoch': 4.0}

In [7]:
trainer.save_model("../../models/intentClassifier")
tokenizer.save_pretrained("../../models/intentClassifier")

('../../models/intentClassifier\\tokenizer_config.json',
 '../../models/intentClassifier\\special_tokens_map.json',
 '../../models/intentClassifier\\vocab.txt',
 '../../models/intentClassifier\\added_tokens.json')

In [8]:
# Inference example
def predict_intent(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
        predicted_class_id = outputs.logits.argmax().item()
    return intent_labels[predicted_class_id]

In [9]:
# Example usage
print(predict_intent("How do I apply for scholarships?"))
print(predict_intent("I have a complaint about the cafeteria service?"))
print(predict_intent("Are there any upcoming student events"))

Scholarship/Financial Aid
General Complaint
Extracurricular Activities


In [10]:
print(predict_intent("When is the tuition payment deadline?"))
print(predict_intent("Hi, I'm trying to figure out how to pay my tuition fees."))

Tuition/Fees Inquiry
Tuition/Fees Inquiry
